In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

from Extract_Observation_bt import extract_observation_bt
from Extract_Simulated_bt import extract_simulated_bt
from Extract_Observation_Rh import extract_observation_rh
from Extract_Inversion_Rh import extract_inversion_rh

In [2]:

# 加载神经网路模型
model_path = r'I:\Data\Personal_Data\graduation_project\Code\Machine_Learning\model_Rh'
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                60        
_________________________________________________________________
dense_2 (Dense)              (None, 47)                517       
Total params: 607
Trainable params: 607
Non-trainable params: 0
_________________________________________________________________


In [3]:
# 将模拟亮温的00和12修改为08和20
def trans_time(i):
    if i[-2:] == '00':
        m = '08'
    elif i[-2:] == '12':
        m = '20'
    return i[:-2] + m

In [4]:
years = ['2007']

# 加载观测亮温，载入模型
o_bt = extract_observation_bt(years)

o_bt.drop(o_bt.columns[:9], axis=1, inplace=True)
o_bt.drop(o_bt.columns[5:], axis=1, inplace=True)

In [5]:
# 归一化
def norm(x):
    return (x - x.describe().transpose()['mean']) / x.describe().transpose()['std']
o_bt = norm(o_bt)

In [6]:
# 加载模拟亮温，载入模型
s_bt = extract_simulated_bt(years)
s_bt.drop(s_bt.columns[5:], axis=1, inplace=True)
s_bt = norm(s_bt).round(3)
s_bt.index = [trans_time(i) for i in s_bt.index]

In [7]:
# 加载榆中站观测数据
o_rh = extract_observation_rh(years)
o_rh.columns = [str((int(i)-1965)/1000) for i in o_rh.columns]
o_rh = o_rh.round(3)
o_rh = o_rh*100
o_rh.index = [trans_time(i) for i in o_rh.index]

In [8]:
# 加载微波辐射计反演数据
i_rh = extract_inversion_rh(years)
# i_rh = pd.to_numeric(i_rh)

In [9]:
print(f'o_bt shape is {o_bt.shape}')
print(f's_bt shape is {s_bt.shape}')
print(f'o_rh shape is {o_rh.shape}')
print(f'i_rh shape is {i_rh.shape}')

o_bt shape is (730, 5)
s_bt shape is (723, 5)
o_rh shape is (631, 47)
i_rh shape is (377, 47)


In [10]:
def Cross_validation(x1, x2):
    for i in x1.index:
        if i not in x2.index:
            x1.drop(i, inplace=True)
    for i in x2.index:
        if i not in x1.index:
            x2.drop(i, inplace=True)

In [11]:
Cross_validation(o_rh, i_rh)

In [12]:
o_bt = o_bt.loc[i_rh.index]
o_bt = norm(o_bt)

s_bt = s_bt.loc[i_rh.index]
s_bt = norm(s_bt)

o_rh = o_rh.loc[i_rh.index]

In [13]:
print(f'o_bt shape is {o_bt.shape}')
print(f's_bt shape is {s_bt.shape}')
print(f'o_rh shape is {o_rh.shape}')
print(f'i_rh shape is {i_rh.shape}')

o_bt shape is (329, 5)
s_bt shape is (329, 5)
o_rh shape is (329, 47)
i_rh shape is (329, 47)


In [14]:
cols = ['o_bt o_rh', 's_bt o_rh', 'i_rh o_rh']
cc = pd.DataFrame(columns=cols)

def draw(i):
    # Height = i_rh.columns
    # plt.figure(figsize=(5,10))
    X1 = model.predict(pd.DataFrame(o_bt.loc[i]).T).T
    X2 = model.predict(pd.DataFrame(s_bt.loc[i]).T).T
    X3 = pd.to_numeric(i_rh.loc[i]).T

    # plt.plot(X1, Height, label='o_bt', linestyle=':')
    # plt.plot(X2, Height, label='s_bt', linestyle='-')
    # plt.plot(o_rh.loc[i], Height, label='o_rh', linestyle='dashdot')
    # plt.plot(i_rh.loc[i], Height, label='i_rh', linestyle='dotted')
    #
    # plt.legend()
    # plt.grid()
    # plt.xlabel('Rh %')
    # plt.ylabel('Altitude km')
    # plt.title(i)
    # path = r'I:\Data\Personal_Data\graduation_project\Graduation_thesis\Atlas\Chapter6\Rh\\'
    # plt.savefig(path+i.replace('/', '').replace(':', '')+'.png',
    #             bbox_inches = 'tight')
    # plt.show()

    # print(f'o_bt 和 o_rh 相关系数为{np.corrcoef(X1.T, o_rh.loc[i].T)[0, 1].round(4)}')
    # print(f's_bt 和 o_rh 相关系数为{np.corrcoef(X2.T, o_rh.loc[i].T)[0, 1].round(4)}')
    # print(f'i_rh 和 o_rh 相关系数为{np.corrcoef(X3, o_rh.loc[i].T)[0, 1].round(4)}')

    cc.loc[i] = [np.corrcoef(X1.T, o_rh.loc[i].T)[0, 1].round(4),
                 np.corrcoef(X2.T, o_rh.loc[i].T)[0, 1].round(4),
                 np.corrcoef(X3, o_rh.loc[i].T)[0, 1].round(4)]

# i = i_rh.index[random.randrange(0, i_rh.shape[0])]
for i in i_rh.index:
    if (i in s_bt.index)and(i in o_rh.index)and(i in o_bt.index):
        # print(i)
        draw(i)

In [20]:
print(cc.mean().round(4))
print(cc.shape)
print(cc.columns)
print((cc['s_bt o_rh']>cc['i_rh o_rh']).sum())

o_bt o_rh    0.1120
s_bt o_rh    0.4876
i_rh o_rh    0.4785
dtype: float64
(329, 3)
Index(['o_bt o_rh', 's_bt o_rh', 'i_rh o_rh'], dtype='object')
161
